In [1]:
FI_API_KEY="d1533c4e62b64efd926355e22350cd11"
FI_SECRET_KEY="a508a3c6cf7647cda67197af574d2b45"

In [2]:
from fi.prompt.client import PromptClient
from fi.prompt.types import ModelConfig, PromptTemplate, SystemMessage, UserMessage

/Users/apple/miniconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "RequestConfig" shadows an attribute in parent "BaseModel"
  warnings.warn(


### Create a prompt from scratch

In [38]:
system_prompt = "You are a customer service assistant, help customers answer their queries"
user_prompt1 = '''
You are a Tier-1 customer support agent for {{product_line}}.  
**Task:**  
1. Classify the customer's issue into one of these categories:  
   - Password reset  
   - Account unlock  
   - Basic troubleshooting (e.g., {{error_code}})  
   - Billing inquiry  
   - Escalate to Tier-2  

2. **If the issue is routine (first three categories):**  
   - Respond in a {{customer_tone}} tone.  
   - Provide a step-by-step resolution using ONLY approved knowledge base articles.  
   - Include:  
     - Clear instructions (max 5 steps).  
     - Safety disclaimer: "Do not share personal data in public forums."  
     - Closing question: "Does this resolve your issue?"  

3. **If escalation is needed:**  
   - Acknowledge urgency: "I'll prioritize this with our specialists."  
   - List NEXT STEPS: "Expect a response within 1 hour via {{preferred_contact_method}}."  

**Rules:**  
- Never invent solutions outside the knowledge base.  
- Use bullet points for readability.  
- Mirror the customer's language style (formal/casual).  
'''
user_prompt2 = "{{userquery}}"

In [39]:
def create_multiple_variables_prompt():
    template = PromptTemplate(
        name="csat_agent",
        messages=[
            SystemMessage(content=system_prompt),
            UserMessage(content=user_prompt1),
            UserMessage(content=user_prompt2)
        ],
        model_configuration=ModelConfig(
            model_name="gpt-4o-mini",
            temperature=0.7
        )
    )
    
    client = PromptClient(template=template, fi_api_key=FI_API_KEY, fi_secret_key=FI_SECRET_KEY, fi_base_url="https://dev.api.futureagi.com")
    return client.create()

In [40]:
if __name__ == "__main__":    
    multi_var_client = create_multiple_variables_prompt()
    response = multi_var_client.run(variables={
        "product_line": "Phone",
        "error_code": "400",
        "customer_tone": "Polite",
        "preferred_contact_method": "Email",
        "userquery": "Phone not starting"
    })

### Generate a prompt

In [7]:
from fi.prompt.client import PromptClient
from fi.prompt.types import ModelConfig, PromptTemplate, SystemMessage, UserMessage

def generate_prompt_content():
    template = PromptTemplate(
        name="csat_generate_prompt",
        messages=[
            SystemMessage(content="You are a customer service representative."),
            UserMessage(content="How can I help you with your {issue}?")
        ],
        model_configuration=ModelConfig(
            model_name="gpt-4o-mini",
            temperature=0.7,
            max_tokens=1000
        )
    )

    prompt_client = PromptClient(template=template, fi_api_key=FI_API_KEY, fi_secret_key=FI_SECRET_KEY, fi_base_url="https://dev.api.futureagi.com")
    
    client = prompt_client.create()
    
    requirements = [
        "Add a friendly greeting at the start",
        "Include empathy in the response",
        "Ask for specific details about the issue"
    ]
    
    for requirement in requirements:
        client = client.generate(requirement)
        print(f"\nGenerated content for requirement: {requirement}")
        print(f"Latest message: {client.template.messages[-1].content}")
    
    return client

if __name__ == "__main__":
    try:
        generated_client = generate_prompt_content()
        print("\nFinal template messages:")
        for msg in generated_client.template.messages:
            print(f"{msg.role}: {msg.content}")
    except Exception as e:
        print(f"Error generating prompt content: {str(e)}")


Generated content for requirement: Add a friendly greeting at the start
Latest message: You are an AI assistant tasked with adding a friendly greeting to the beginning of a given text. Your goal is to create a welcoming and positive first impression while maintaining the original content's integrity.

Follow these steps to complete the task:

1. Read the provided text carefully:
<original_text>
{{ORIGINAL_TEXT}}
</original_text>

2. Choose a friendly, generic greeting that can work in multiple contexts. Some options include:
   - "Hello there!"
   - "Hi, welcome!"
   - "Greetings!"
   - "Hey, nice to see you!"

3. Add the chosen greeting at the very beginning of the text, followed by a space.

4. Ensure the greeting ends with an exclamation mark to convey friendliness and enthusiasm.

5. Keep the greeting short and simple (2-4 words) to accommodate various potential use cases.

6. Maintain the original content exactly as provided, without any modifications.

Now, combine the chosen gr

### Improve an Existing Prompt

In [ ]:
from fi.prompt.client import PromptClient
from fi.prompt.types import ModelConfig, PromptTemplate, SystemMessage, UserMessage

def improve_prompt_examples():
    """Examples of improving prompts with different approaches"""

    template = PromptTemplate(
        name="customer_support_template",
        messages=[
            SystemMessage(content="You are a customer service representative."),
            UserMessage(content="How can I help with your {issue_type} today?")
        ],
        model_configuration=ModelConfig(
            model_name="gpt-4",
            temperature=0.7,
            max_tokens=1000
        )
    )

    prompt_client = PromptClient(template=template, fi_api_key=FI_API_KEY, fi_secret_key=FI_SECRET_KEY, fi_base_url="https://dev.api.futureagi.com")
    client = prompt_client.create()

    # Example 1: Basic Improvement - Make Language More Professional
    client = client.improve("Make the language more formal and professional")
    print("\nAfter professional improvement:")
    print(f"Latest message: {client.template.messages[-1].content}")

    # Example 2: Improve Clarity and Structure
    client = client.improve(
        "Add clear structure with greeting, issue identification, and next steps"
    )
    print("\nAfter structural improvement:")
    print(f"Latest message: {client.template.messages[-1].content}")

    # Example 3: Add Empathy and Personalization
    client = client.improve(
        "Add empathetic language and personalization while maintaining professionalism"
    )
    print("\nAfter empathy improvement:")
    print(f"Latest message: {client.template.messages[-1].content}")

    # Example 4: Optimize for Specific Use Case
    client = client.improve(
        "Optimize for technical support issues with relevant terminology"
    )
    print("\nAfter use-case optimization:")
    print(f"Latest message: {client.template.messages[-1].content}")

    return client

if __name__ == "__main__":
    try:
        improved_client = improve_prompt_examples()
        print("\nFinal improved template:")
        for msg in improved_client.template.messages:
            print(f"{msg.role}: {msg.content}")
    except Exception as e:
        print(f"Error improving prompt: {str(e)}")